<a href="https://colab.research.google.com/github/passtock/stock-straregy/blob/main/stock_7_209_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np

def get_stock_price_data(stock_symbol, start_date, end_date):
    stock_data = yf.download(stock_symbol, start=start_date, end=end_date)
    return stock_data

def calculate_moving_average(data, window):
    return data['Close'].rolling(window=window).mean()

def simulate_trading(stock_data, short_window, long_window, initial_cash):
    stock_data = stock_data.copy()  # 원본 데이터 보호
    stock_data['Short_MA'] = calculate_moving_average(stock_data, short_window)
    stock_data['Long_MA'] = calculate_moving_average(stock_data, long_window)
    stock_data['Signal'] = 0  # 1 for buy, -1 for sell

    position = 0  # 0 means no position, 1 means holding stock
    cash = initial_cash
    holdings = 0  # Number of shares held
    cash_history = []

    for i in range(1, len(stock_data)):
        if stock_data['Short_MA'].iloc[i] > stock_data['Long_MA'].iloc[i] and stock_data['Short_MA'].iloc[i - 1] <= stock_data['Long_MA'].iloc[i - 1]:
            if position == 0:  # Buy signal
                price = stock_data['Close'].iloc[i]
                holdings = (cash / price) * 0.995  # Subtract 0.5% fee
                cash = 0
                position = 1
                stock_data.at[stock_data.index[i], 'Signal'] = 1

        elif stock_data['Short_MA'].iloc[i] < stock_data['Long_MA'].iloc[i] and stock_data['Short_MA'].iloc[i - 1] >= stock_data['Long_MA'].iloc[i - 1]:
            if position == 1:  # Sell signal
                price = stock_data['Close'].iloc[i]
                cash = holdings * price * 0.995  # Subtract 0.5% fee
                holdings = 0
                position = 0
                stock_data.at[stock_data.index[i], 'Signal'] = -1

        # Update total cash value considering current holdings
        total_value = cash + holdings * stock_data['Close'].iloc[i]
        cash_history.append(total_value)

    # cash_history 길이가 stock_data와 일치하도록 조정
    if len(cash_history) < len(stock_data):
        last_value = cash_history[-1] if cash_history else initial_cash
        cash_history += [last_value] * (len(stock_data) - len(cash_history))

    stock_data['Total Value'] = cash_history
    return stock_data

def optimize_moving_average(stock_data, short_windows, long_windows, initial_cash):
    best_strategy = None
    best_return = -np.inf  # 최악의 경우를 초기화
    results = []

    for short_window in short_windows:
        for long_window in long_windows:
            if long_window - short_window >= 30:
                if short_window >= long_window:
                    continue  # 짧은 이평선은 긴 이평선보다 짧아야 함

                simulated_data = simulate_trading(stock_data, short_window, long_window, initial_cash)
                final_value = simulated_data['Total Value'].iloc[-1]
                total_return = final_value - initial_cash

                results.append((short_window, long_window, total_return))

                if total_return > best_return:
                    best_return = total_return
                    best_strategy = (short_window, long_window)

    return best_strategy, results

def plot_interactive_chart(stock_data):
    # Create subplots
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.2,
                        row_heights=[0.7, 0.3])

    # Stock price chart
    fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close'],
                             mode='lines', name='Stock Price', line=dict(color='blue')),
                  row=1, col=1)

    # Portfolio value chart
    fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Total Value'],
                             mode='lines', name='Portfolio Value', line=dict(color='black')),
                  row=2, col=1)

    # Update layout for better visuals
    fig.update_layout(title=f'{stock_data.index[-1].strftime("%Y-%m-%d")} Stock Price and Portfolio Value',
                      yaxis_title='Price',
                      xaxis_title='Date',
                      yaxis2_title='Portfolio Value',
                      xaxis_rangeslider_visible=False,
                      showlegend=True)

    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)

    fig.show()

def main():
    stock_symbol = 'TQQQ'  # 예시로 TQQQ 주식을 사용합니다.
    start_date = '2019-01-01'  # 시작 날짜
    end_date = '2024-08-15'  # 종료 날짜
    initial_cash = 10000  # 초기 자금

    stock_data = get_stock_price_data(stock_symbol, start_date, end_date)

    short_windows = range(1, 210)  # 1일에서 50일 사이의 이평선
    long_windows = range(1, 210)   # 1일에서 50일 사이의 이평선

    best_strategy, results = optimize_moving_average(stock_data, short_windows, long_windows, initial_cash)

    if best_strategy is not None:
        print(f"최고의 투자 전략: 단기 이평선 {best_strategy[0]}일, 장기 이평선 {best_strategy[1]}일")
        print(f"최종 수익: {best_strategy[0]}일, {best_strategy[1]}일 이평선의 최종 수익 = {results}")

        # 최적의 전략으로 다시 시뮬레이션 실행 및 그래프 출력
        short_window, long_window = best_strategy
        stock_data = simulate_trading(stock_data, short_window, long_window, initial_cash)

        plot_interactive_chart(stock_data)
    else:
        print("유효한 전략을 찾을 수 없습니다.")

if __name__ == "__main__":
    main()


[*********************100%%**********************]  1 of 1 completed


최고의 투자 전략: 단기 이평선 7일, 장기 이평선 209일
최종 수익: 7일, 209일 이평선의 최종 수익 = [(1, 31, 18634.93996865996), (1, 32, 20608.731870225973), (1, 33, 17986.45613191603), (1, 34, 14959.559268883342), (1, 35, 17293.08458550968), (1, 36, 19910.314780509892), (1, 37, 22067.58557752191), (1, 38, 20528.610307523388), (1, 39, 16120.281505793628), (1, 40, 11639.772314156562), (1, 41, 15185.479741850097), (1, 42, 14870.641060103899), (1, 43, 21287.70009302436), (1, 44, 20923.97406550727), (1, 45, 22742.71136673182), (1, 46, 21417.98870535685), (1, 47, 19228.684063380424), (1, 48, 28424.396347406815), (1, 49, 24457.092936664558), (1, 50, 15425.17153396285), (1, 51, 7891.979822033791), (1, 52, 9179.678640867434), (1, 53, 4843.740869644886), (1, 54, 3180.8220813895314), (1, 55, 2503.026270472392), (1, 56, 995.6169195053153), (1, 57, 3241.8135810100575), (1, 58, 4818.773644956456), (1, 59, 4673.038710441497), (1, 60, 7322.438186105552), (1, 61, 8455.237841580736), (1, 62, 7105.822087639768), (1, 63, 4010.216269949253),